# Topic Selection and Postprocessing

In [1]:
%matplotlib inline
import pickle
from os import listdir, makedirs
from os.path import join, isfile, exists, dirname
import gc
import re
import math

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%pylab inline
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import TfidfModel, LdaModel
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from tqdm._tqdm_notebook import tqdm_notebook

from constants import *
from utils import load, init_logging, tprint, TopicsLoader
from topic_reranking import Reranker

#from eval_lda import eval_coherence

tqdm_notebook.pandas()

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.options.display.max_columns = 80
pd.options.display.max_rows = 2000
pd.options.display.precision = 3

-----

## Joining Topics / Labels / Scores

In [3]:
phra = load('phrases', 'lemmap')
wikt = load('wikt', 'lemmap')

fLoading {file}


In [4]:
args = ['e42', 100]
metrics = ['c_npmi_wikt', 'pairwise_similarity_ftx']

In [39]:
def combine(dataset, min_regular_terms=6, min_cnpmi=0, min_pairsim=0.3):
    topics = load(dataset, 'topics', *args)
    labels_w2v = load(dataset, 'labels', *args, 'w2v', 'minimal').rename(columns=lambda x: x + '_w2v').reset_index(level='label_method', drop=True)
    labels_ftx = load(dataset, 'labels', *args, 'ftx', 'minimal').rename(columns=lambda x: x + '_ftx').reset_index(level='label_method', drop=True)
    scores = load(dataset, 'scores', *args, 'ftx', 'minimal')[metrics]

    scores['rank_cnpmi'] = scores.c_npmi_wikt.rank()
    scores['rank_pairsim'] = scores.pairwise_similarity_ftx.rank()
    scores['rank_mean'] = (scores.rank_cnpmi + scores.rank_pairsim) / 2
    col_mean = scores[metrics].mean()
    col_stdv = scores[metrics].std()
    print(col_mean)
    scores_norm = (scores[metrics] - col_mean) / col_stdv
    scores_norm = scores_norm.rename(columns=lambda x: x + '_norm')
    scores_norm['avg'] = scores_norm.mean(axis=1)
    scores = scores.join(scores_norm)
    scores['avg_rank'] = scores.avg.rank()
    print('average difference per topic between the mean of the individual ranks and the rank of the normalized mean:', (scores.rank_mean - scores.avg_rank).abs().sum() / len(scores))
    # an advantage of ranking by taking the normalized mean: minimal chance of equal ranks -> disadvantage: metric is not comparable to other datasets anymore -> use metrics as a filter

    df = pd.concat([topics, labels_w2v, labels_ftx, scores], axis=1)
    df['count_regular_terms'] = df.loc[:, 'term0':'term9'].applymap(lambda x: x in wikt.index).sum(axis=1)
    df = df.query('count_regular_terms >= @min_regular_terms and c_npmi_wikt >= @min_cnpmi and pairwise_similarity_ftx >= @min_pairsim')
    print(len(df))
    df = df.sort_values('avg_rank', ascending=False)
    return df

In [44]:
combine('dewac', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)

Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-candidates.csv
File b'../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-candidates.csv' does not exist
Loading topics via TopicsLoader
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.123
pairwise_similarity_ftx    0.457
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.12
72


term0           term1  \
dataset param_id nb_topics topic_idx                                 
dewac   e42      100       36               Montag         Freitag   
                           62         Arbeitnehmer     Arbeitgeber   
                           56         Studierender         Studium   
                           78                 Arzt         Patient   
                           26               Schule         Schüler   
                           85                  Dr.           Prof.   
                           39                 Frau            Mann   
                           53                 Salz          Minute   
                           4                 Musik            Band   
                           47                 Tier            Hund   
                           90                 Kind           Elter   
                           71                Spiel      Mannschaft   
                           18               Kirche        Religion   
                           25                 Euro         Prozent   
                           38                 Wahl         Sitzung   
                           33             Mitglied          Verein   
                           69          Ausstellung           Kunst   
                           55               Körper      Erkrankung   
                           51             Computer        Programm   
                           82               Kirche        Gemeinde   
                           11                Roman           Leben   
                           93                 Auto        Fahrzeug   
                           65                 Abs.          Urteil   
                           95                Stadt             Ort   
                           6               Polizei           Opfer   
                           45                 Erde          Planet   
                           16                Media       Fernsehen   
                           86              Sprache         Deutsch   
                           9                 Stoff           Menge   
                           99                Kunde         Angebot   
                           43          Unternehmen           Markt   
                           29                  Zug            Bahn   
                           60                Krieg          Soldat   
                           13                 Film            Kino   
                           3                  Abs.            Satz   
                           40              Pflanze            Baum   
                           52                 Gott            Herr   
                           28           Ausbildung     Mitarbeiter   
                           31                Recht             Art   
                           67                 Fall         Richter   
                           73                 Hand            Auge   
                           27              Begriff            Form   
                           66                Platz           Sport   
                           87                Seite            Link   
                           54              Politik          Partei   
                           5           Entwicklung            Ziel   
                           34                Spiel           Punkt   
                           76              Gebäude            Haus   
                           68               E-Mail            Tel.   
                           37              Prüfung       Zulassung   
                           80               Reform            Land   
                           44              Projekt           Thema   
                           30           Flüchtling     Deutschland   
                           72                Daten     Information   
                           41          Deutschland            Jude   
                           15             Leistung     

In [41]:
combine('dewac1', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)

Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_noun_bow_topic-candidates.csv
File b'../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_noun_bow_topic-candidates.csv' does not exist
Loading topics via TopicsLoader
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac1_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac1_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac1_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.079
pairwise_similarity_ftx    0.402
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 1.72
57


term0          term1  \
dataset param_id nb_topics topic_idx                                       
dewac1  e42      100       94                      Schule        Schüler   
                           6                       Kirche        Bischof   
                           24                        Tier           Hund   
                           12                       Musik           Band   
                           72                     Patient           Arzt   
                           69                        Buch          Autor   
                           9                  Zivildienst    Wehrpflicht   
                           22                        Frau          Vater   
                           66                    Mitglied         Verein   
                           45                         Uhr          Quell   
                           40                        Kind          Elter   
                           60                  Ausbildung        Studium   
                           39                  Mathematik         Lehrer   
                           35                    Programm       Computer   
                           7                 Entscheidung      Verfahren   
                           4                 Gesellschaft   Aufsichtsrat   
                           87                     Polizei           Mann   
                           2          Lebenspartnerschaft  Lebenspartner   
                           90                        Gott          Jesus   
                           19                  Richtlinie       Regelung   
                           83                     Prozent           Euro   
                           79                      Wasser         Minute   
                           0                       Partei            PDS   
                           31                       Seite    Information   
                           67                        Haus          Stadt   
                           38                      Mensch        Begriff   
                           14                 Unternehmen          Kunde   
                           30                 Deutschland           Jude   
                           73                     Vertrag           Ware   
                           93                     Projekt          Daten   
                           70                 Jugendliche    Einrichtung   
                           26                        Land          Stadt   
                           5                      Artikel         Gesetz   
                           32                         USA          Krieg   
                           49                Arbeitsplatz           Euro   
                           97                     Artikel         Gesetz   
                           99                      Studie       Ergebnis   
                           96                      Mensch        Pflanze   
                           18                       Leute           Zeit   
                           50                      Mensch          Staat   
                           25                      Mensch          Leben   
                           64                  Angeklagte           Fall   
                           27                        Wahl       Gemeinde   
                           46                     Sendung           Tier   
                           56               Veranstaltung            Uhr   
                           36                     Kollege         Arbeit   
                           59                         Tag            Uhr   
                           21                       Spiel        Spieler   
                           65                       Kunst           Werk   
                           68                     Artikel           Erde   
                           17                Rechtsanwalt           Erbe   
                        

In [42]:
combine('n', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)

Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_topic-candidates.csv
File b'../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_topic-candidates.csv' does not exist
Loading topics via TopicsLoader
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/news_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/news_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.095
pairwise_similarity_ftx    0.416
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 1.92
60


term0         term1  \
dataset param_id nb_topics topic_idx                              
news    e42      100       19          Smartphone        Google   
                           67                Song          Band   
                           12             Prozess          Haft   
                           20                Wahl        Partei   
                           7                 Kind       Familie   
                           15               Preis     Kategorie   
                           27            Facebook      Internet   
                           72             Polizei          Mann   
                           74                 ZDF        Sender   
                           2                 Werk      Künstler   
                           96                Euro       Million   
                           80               Apple         Gerät   
                           92                Film  Schauspieler   
                           97            Flugzeug          Flug   
                           9                 Auto      Fahrzeug   
                           83             Anleger        Dollar   
                           5                 Arzt       Patient   
                           87                Fall        Urteil   
                           89              Kirche         Papst   
                           17         Arbeitgeber  Gewerkschaft   
                           86                Auto        Unfall   
                           8          Unternehmen     Milliarde   
                           60               Daten        E-Mail   
                           1                Kunde       Händler   
                           29                Show       Staffel   
                           58                Haus       Wohnung   
                           65                Bank          Geld   
                           23              Gesetz           AfD   
                           81                Buch    Geschichte   
                           47           Eintracht       Trainer   
                           32            Mitglied   Deutschland   
                           56               König         Farbe   
                           35               Strom         Tonne   
                           59              Schule       Schüler   
                           64          Flüchtling   Deutschland   
                           18                Foto          Bild   
                           70           Milliarde          Euro   
                           39              Europa          Iran   
                           73                Frau          Mann   
                           98                Zeit           Tag   
                           57             Protest   Demonstrant   
                           88            Anschlag        Mensch   
                           31               Frage       Antwort   
                           76              Mensch        Studie   
                           49               Stück         Bühne   
                           42               Schuh          Mode   
                           14             Bericht       Vorwurf   
                           62             Prozent       Umfrage   
                           36         Unternehmen   Mitarbeiter   
                           77              Mensch         Leben   
                           99         Deutschland       Behörde   
                           46          Restaurant          Gast   
                           79             Polizei       Beamter   
                           41                 Uhr        Stunde   
                           53              Mensch         Frage   
                           55               China           USA   
                           3                Meter          Reis   
                           75         Deutschland          Zahl   
                        

In [43]:
combine('s', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)

Reading ../data/preprocessed/LDAmodel/noun/bow/topics/speeches_noun_bow_topic-candidates.csv
File b'../data/preprocessed/LDAmodel/noun/bow/topics/speeches_noun_bow_topic-candidates.csv' does not exist
Loading topics via TopicsLoader
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/speeches_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/speeches_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/speeches_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/speeches_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/speeches_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/speeches_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.069
pairwise_similarity_ftx    0.387
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.14
54


term0             term1  \
dataset  param_id nb_topics topic_idx                                       
speeches e42      100       54                     Film              Kino   
                            97             Wissenschaft       Universität   
                            36                   Bürger             Recht   
                            64          Änderungsantrag        Kommission   
                            90                   Abfall      Lebensmittel   
                            32                  Bericht                EU   
                            71                     Kind          Internet   
                            44                    Opfer              Frau   
                            69               Verfassung           Gericht   
                            68                Parlament               Rat   
                            47                Flughafen  Fluggesellschaft   
                            39                     Kind            Mensch   
                            43                  Energie        Kommission   
                            77            Menschenrecht         Präsident   
                            53                Präsident  europäisch_Union   
                            28              Katastrophe  Naturkatastrophe   
                            70               Kommission         Parlament   
                            66                Parlament             Daten   
                            8                       Rat             Frage   
                            50                Präsident         Parlament   
                            49               Richtlinie         Vorschlag   
                            74                  Patient            Mensch   
                            24            Mitgliedstaat     Freizügigkeit   
                            31              Binnenmarkt     Mitgliedstaat   
                            22                   Türkei  europäisch_Union   
                            57              Präsidentin              Frau   
                            92             Gesellschaft           Bildung   
                            42                   Kultur           Sprache   
                            81            Mitgliedstaat               Rat   
                            91               Kommission     Mitgliedstaat   
                            58              Deutschland    Zusammenarbeit   
                            61           Landwirtschaft          Landwirt   
                            2                     Frage           Politik   
                            98                   Kultur             Kunst   
                            62         Entwicklungsland             Armut   
                            41             Arbeitnehmer       Unternehmen   
                            37                Präsident        Kommission   
                            85                    Media             Stadt   
                            12               Kommission            Europa   
                            88          Änderungsantrag        Abstimmung   
                            56                     Frau           Bericht   
                            45                    China            Region   
                            96                      Rat         Parlament   
                            25                   Europa  europäisch_Union   
                            78              Deutschland            Mensch   
                            0                       Rom         Rassismus   
                            89                     Euro           Währung   
                            7                    Europa             Krise   
                            35               Kommission           Bericht   
                            21                 Programm        Kommission   
                            59            Kern

In [47]:
combine('o', min_regular_terms=6, min_cnpmi=-0.1, min_pairsim=0.3)

Reading ../data/preprocessed/LDAmodel/noun/bow/topics/OnlineParticipation_noun_bow_topic-candidates.csv
File b'../data/preprocessed/LDAmodel/noun/bow/topics/OnlineParticipation_noun_bow_topic-candidates.csv' does not exist
Loading topics via TopicsLoader
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/OnlineParticipation_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/OnlineParticipation_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/OnlineParticipation_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/OnlineParticipation_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/OnlineParticipation_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/OnlineParticipation_noun_bow_topic-scores_germanet.csv
c_npmi_wikt               -0.021
pairwise_similarity_ftx    0.318
dtype: float64
average difference per topic between the m

term0  \
dataset             param_id nb_topics topic_idx                  
OnlineParticipation e42      100       94            Autofahrer   
                                       90             Radfahrer   
                                       42                  Kind   
                                       19                Radweg   
                                       68         Fahrradfahrer   
                                       96              Richtung   
                                       11               Gebäude   
                                       79                  Auto   
                                       82                  ÖPNV   
                                       59         Beschilderung   
                                       2              Vorschlag   
                                       66         Zebrastreifen   
                                       23                  Euro   
                                       7                Nutzung   
                                       95                  Haus   
                                       25                 Stadt   
                                       83                   Rat   
                                       15              Haushalt   
                                       8                Schüler   
                                       72                  Geld   
                                       50               Wohnung   
                                       26                  Hund   
                                       28                 Platz   
                                       4                   Köln   
                                       62            Spielplatz   
                                       13            Schlagloch   
                                       35            Moderation   
                                       16                  Bonn   
                                       65               Bahnhof   
                                       55              Programm   
                                       3                  Seite   
                                       92                   Bus   
                                       14                Lösung   
                                       1                 Arbeit   
                                       37                   PKW   
                                       38               Beitrag   
                                       54                   Ort   
                                       81                Fläche   
                                       56                Antrag   
                                       17            Stadt_Köln   
                                       34                   EUR   
                                       97                Gehweg   
                                       6                   Idee   
                                       20           Deutschland   
                                       60                Schild   
                                       88                Anfang   
                                       74                   Bad   
                                       64              Übergang   
                                       89                  Bahn   
                                       21           Unternehmen   
                                       49                Verein   

                                                           term1  \
dataset             param_id nb_topics topic_idx                   
OnlineParticipation e42      100       94              Radfahrer   
                                       90              Fußgänger   
                                       42                 Schule   
                                       19                    Weg   
                                       68          Einbahnstraße   
                              